In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install  bokeh --quiet

In [ ]:
!pip install -U pathway

In [ ]:
!pip install panel

In [ ]:
import pathway as pw
import datetime
from datetime import datetime
import bokeh.plotting
import panel as pn

Model 2: Demand-Based Price Function

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/dataset.csv')
df_1

In [ ]:
df_1.SystemCodeNumber.unique()

In [ ]:
df_1

In [ ]:
df_1['Timestamp'] = pd.to_datetime(df_1['LastUpdatedDate'] + ' ' + df_1['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
df_1.head()

In [ ]:
df_1.VehicleType.unique()

In [ ]:
df_1.TrafficConditionNearby.unique()

In [ ]:
df_1['TrafficConditionNearby']=df_1['TrafficConditionNearby'].replace({'low':0, 'average':1, 'high':2})

In [ ]:
 df_1['VehicleType']=df_1['VehicleType'].replace({'cycle':0,'car':2, 'bike':1, 'truck':3})

In [ ]:
df_1

In [ ]:
df_1.dtypes

In [ ]:
df_1=df_1.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
df_1[['Timestamp','Occupancy','Capacity','SystemCodeNumber','VehicleType','IsSpecialDay','TrafficConditionNearby','QueueLength']].to_csv('park5_stream.csv',index=False)

In [ ]:
import pathway as pw
class park5_schema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int  # Number of occupied parking spots
    Capacity: int
    SystemCodeNumber: str
    VehicleType: int
    IsSpecialDay: int
    TrafficConditionNearby: int
    QueueLength: int


In [ ]:
data_1 = pw.demo.replay_csv("park5_stream.csv", schema=park5_schema, input_rate=1000)

In [ ]:
fmt='%Y-%m-%d %H:%M:%S'
data_1_with_time = data_1.with_columns(t=data_1.Timestamp.dt.strptime(fmt),day = data_1.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [ ]:
import datetime


delta_window_1=data_1_with_time.windowby(pw.this.t,instance=pw.this.SystemCodeNumber,window=pw.temporal.tumbling(datetime.timedelta(days=1)),
behavior=pw.temporal.exactly_once_behavior()).with_columns(demand = 2*(pw.this.Occupancy/pw.this.Capacity) - (0.5*pw.this.QueueLength) - (0.5*pw.this.TrafficConditionNearby) +
                                                    pw.this.IsSpecialDay + (0.5*pw.this.VehicleType)).select(pw.this.SystemCodeNumber,
                                               pw.this.t,pw.this.demand)

pw.debug.compute_and_print(delta_window_1)


In [ ]:
reduced_table = delta_window_1.reduce(min_demand=pw.reducers.min(delta_window_1.demand),max_demand=pw.reducers.max(delta_window_1.demand))


In [ ]:
min_max_demand_table = delta_window_1.select(
    *pw.this, reduced_table.ix_ref().min_demand, reduced_table.ix_ref().max_demand
)


In [ ]:
normalized_table = min_max_demand_table.with_columns(
    normalized_demand=(
        (pw.this.demand - pw.this.min_demand) /
        (pw.this.max_demand - pw.this.min_demand + 1e-6)  # avoid divide-by-zero
    )
).select(pw.this.t, pw.this.SystemCodeNumber, pw.this.normalized_demand)


In [ ]:
price_table = normalized_table.windowby(pw.this.t,instance=pw.this.SystemCodeNumber,window=pw.temporal.tumbling(datetime.timedelta(days=1)),
behavior=pw.temporal.exactly_once_behavior()).with_columns(price = 10 + (1 + 2*pw.this.normalized_demand)).select(pw.this.t,
                                    pw.this.SystemCodeNumber,pw.this.price)


In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.scatter("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = price_table.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

In [ ]:
pw.run()

Model 1: Baseline Linear Model

In [ ]:
import pathway as pw
import datetime
from datetime import datetime
import bokeh.plotting
import panel as pn

In [ ]:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/dataset.csv')
df

In [ ]:
df.Latitude.unique()

In [ ]:
df.Longitude.unique()

In [ ]:
df['Location']=str(df['Latitude'])+" "+str( df['Longitude'])

In [ ]:
df

In [ ]:
l=np.array(df.SystemCodeNumber.unique())
l

In [ ]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

In [ ]:
df.head()

In [ ]:
df=df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
df

In [ ]:
type(df)

In [ ]:
grouped_df=df.groupby('SystemCodeNumber',sort=False).apply(lambda x: x ).reset_index(drop=True)

In [ ]:
grouped_df

In [ ]:
grouped_df['price'] = 2*grouped_df['Occupancy']/grouped_df['Capacity']

In [ ]:
grouped_df['prev_price'] = grouped_df['price'].shift(1).fillna(0)

In [ ]:
grouped_df['prev_occ'] = grouped_df['Occupancy'].shift(1).fillna(0)

In [ ]:
grouped_df

In [ ]:
grouped_df[['Timestamp','Occupancy','Capacity','SystemCodeNumber','price','prev_price','prev_occ']].to_csv('park7_stream.csv',index=False)

In [ ]:
class Park7_Schema(pw.Schema):
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: float  # Number of occupied parking spots
    Capacity: float
    SystemCodeNumber: str
    price: float
    prev_price: float
    prev_occ: float


In [ ]:
data = pw.demo.replay_csv("park7_stream.csv", schema=Park7_Schema, input_rate=1000)
pw.debug.compute_and_print(data)

In [ ]:
fmt='%Y-%m-%d %H:%M:%S'
data_with_time=data.with_columns(time=pw.this.Timestamp.dt.strptime(fmt),t=data.Timestamp.dt.strptime(fmt).dt.floor('1d'),
                                 day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00"))
pw.debug.compute_and_print(data_with_time)

In [ ]:

import datetime


delta_window=data_with_time.windowby(pw.this.t,instance=pw.this.SystemCodeNumber,window=pw.temporal.tumbling(datetime.timedelta(days=1)),
behavior=pw.temporal.exactly_once_behavior()).with_columns(final_price=pw.if_else(pw.this.Occupancy > pw.this.prev_occ,
( pw.this.price + pw.this.prev_price + 10.0), (10.0 + pw.this.price - pw.this.prev_price))).select(pw.this.t,pw.this.SystemCodeNumber,
                                                        pw.this.final_price,pw.this.price)
pw.debug.compute_and_print(delta_window)

In [ ]:
import pathway
print(pathway.__version__)


In [ ]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plott(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "final_price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.scatter("t", "final_price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
tem = delta_window.plot(price_plott, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(tem).servable()

In [ ]:
pw.run()